In [1]:
# All imports are in this cell
import re
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
# All constants are in this cell
fileName = 'MachauWingiesChatData.txt'

In [3]:
def getDateTimeNameMessage(line):
    '''This function takes a line as input parameter
    and returns a tuple in the following order
    (date, time, name, message)
    date as datetime type
    time in hh:mm am/pm format as string type'''
    
    date = re.search("[0-9]{2}/[0-9]{2}/[0-9]{4}", line)
    time = re.search("[0-9]+:[0-9]{2}\s[ap]m", line)
    name = re.search("\s-\s(.*?):", line)
    message = re.search("\s-\s.*:\s(.+)", line)
    
    if date is not None and time is not None and name is not None and message is not None:   # This line contains a new data
        date = datetime.strptime(date.group(0), '%d/%m/%Y')
        time = time.group(0)
        name = name.group(1)
        message = message.group(1)
    
    elif date is not None and time is not None and (name is None or message is None):  # Someone left a group message
        date = None
        time = None
        name = None
        message = None
    
    else:                           # This only contains message, continuation of previous message
        date = None
        time = None
        name = None
        message = line
    
    return (date, time, name, message)

In [4]:
def getSimplifiedChatData(filename):
    '''This function takes filename as input parameter
    and returns a list of all chat data
    with each value being a tuple in the order
    (date, time, name, message)
    
    This function merges large messages which come in new line in chat data text file'''
    
    chatDataTxt = open(filename, 'r', encoding="utf8") # opened as read only
    chatDataList = []                 # to store and return the simplified data
    
    for eachLine in chatDataTxt:
        dateTimeNameMsgTuple = getDateTimeNameMessage(eachLine)
        
        if dateTimeNameMsgTuple[0] is not None:          # New data found
            chatDataList.append(dateTimeNameMsgTuple)
        
        elif dateTimeNameMsgTuple[-1] is None:           # Message is None i.e someone left a group. Skip this data
            pass
        
        else:                                            # message continues from previous data
            newMsgForPreviousData = chatDataList[-1][-1] + dateTimeNameMsgTuple[-1]
            dateForPreviousData = chatDataList[-1][0]
            timeForPreviousData = chatDataList[-1][1]
            nameForPreviousData = chatDataList[-1][2]
            
            chatDataList[-1] = (dateForPreviousData, timeForPreviousData, nameForPreviousData, newMsgForPreviousData)
    
    return chatDataList

In [5]:
def getAllParticipantsName(filename, includeCompleteName = False):
    '''This function returns a list of names of all the group participants
    Requirement is that they should have posted atleast a single message
    Name returned is the name saved in persons whose data has been shared
    
    Pass the filename to this function
    Optionally it accepts includeCompleteName parameter which if true returns complete name
    else just first name is included'''
    
    allChatDataSimplified = getSimplifiedChatData(filename)  #Getting simplified data
    allChatParticipants = set()                          #varible of set type to store all participants name
    
    for eachChatData in allChatDataSimplified:
        name = eachChatData[2]
        
        if includeCompleteName:                          #full name is required
            pass
        else:                                            #only first name is required
            name = name.split()[0]                       
        
        allChatParticipants.add(name)
    
    return list(allChatParticipants)  

In [6]:
def GetBasicStats(chatDataList):
    ''' Input: list output from getSimplifiedChatData function
        Output: tuples of general stats (nMsg, ChatDuration, nCharacters, nWords, nMedia, AvgMsgPerDay, AvgCharPerMsg, 
                                        AvgCharPerDay, LenLongestMsg, AvgWordsPerMsg, AvgWordsPerDay, AvgMediaPerDay)
    '''
    
    nMsg = len(chatDataList)
    ChatDuration = (chatDataList[-1][0] - chatDataList[0][0]).days + 1
    
    nCharacters = 0 #including spaces
    nWords = 0
    nMedia = 0
    LenLongestMsg = 0
    
    for item in chatDataList:
        nCharacters = nCharacters + len(item[3])
        nWords = nWords + len(item[3].split())
        if item[3] == "<Media omitted>":
            nMedia = nMedia + 1
        LenLongestMsg = max(LenLongestMsg, len(item[3]))
        
        
    AvgMsgPerDay = int(nMsg/ChatDuration)
    AvgCharPerMsg = int(nCharacters/nMsg)
    AvgCharPerDay = int(AvgMsgPerDay*AvgCharPerMsg)
    AvgWordsPerMsg = int(nWords/nMsg)
    AvgWordsPerDay = int(AvgWordsPerMsg*AvgMsgPerDay)
    AvgMediaPerDay = int(nMedia/ChatDuration)
        
        
    return (nMsg, ChatDuration, nCharacters, nWords, nMedia, AvgMsgPerDay, AvgCharPerMsg, AvgCharPerDay, LenLongestMsg, AvgWordsPerMsg, AvgWordsPerDay, AvgMediaPerDay)

In [7]:
def GetIndividualDataDistribution(chatDataList):
    '''
    Input: list output from getSimplifiedChatData function
    output: Dictionary[First Name as key]: ChatData for that Individual
    '''
    MembersData = {}
    
    
    for item in chatDataList:
        firstName = item[2].split()[0]
        if firstName not in MembersData:
            MembersData[firstName] = []
            
        MembersData[firstName].append(item)
        
    
    
    
    return MembersData
    

In [8]:
def getIndividualBasicStats(chatDataList):
    '''
    Input: list output from getSimplifiedChatData function
    Output: Dictionary[First Name as key]: Basic Stats of chat for that Individual
    '''
    
    MembersData = GetIndividualDataDistribution(chatDataList)
    
    IndividualBasicStats = {}
    
    for keys in MembersData.keys():
        IndividualBasicStats[keys] = GetBasicStats(MembersData[keys])
        
    return IndividualBasicStats

In [9]:
def getDayWiseDataDistribution(chatDataList):
    '''
    Input: list output from getSimplifiedChatData function
    Output: Dictionary[Date in datetime as key]: ChatData for that Date
    '''
    
    DayWiseDistribution = {}
    
    for item in chatDataList:
        if item[0] not in DayWiseDistribution:
            DayWiseDistribution[item[0]] = []
            
        DayWiseDistribution[item[0]].append(item)
        
    return DayWiseDistribution
        
    
    

In [10]:
def getDayWiseBasicStats(chatDataList):
    '''
    Input: list output from getSimplifiedChatData function
    Output: Dictionary[Date in datetime type as key]: Basic Stats of chat for that Date
    '''
    DayWiseDistribution = getDayWiseDataDistribution(chatDataList)
    
    DayWiseBasicStats = {}
    
    for key in DayWiseDistribution.keys():
        DayWiseBasicStats[key] = GetBasicStats(DayWiseDistribution[key])
        
        
    return DayWiseBasicStats

In [11]:
def getDayWisePersonWiseDistribution(chatDataList):
    '''
    Input: list output from getSimplifiedChatData function
    Output: Dictionary[Date in datetime type as key]: Dictionary[First Name as key]: ChatData for that Individual that day
    '''
    
    DayWiseDistribution = getDayWiseDataDistribution(chatDataList)
    
    DayMemberDistribution = {}
    
    for key in DayWiseDistribution.keys():
        DayMemberDistribution[key] = GetIndividualDataDistribution(DayWiseDistribution[key])
        
    return DayMemberDistribution
    

In [12]:
def getDayWisePersonWiseBasicStats(chatDataList):
    '''
    Input: List output from getSimplifiedChatData function
    Output: Dictionary[Date in datetime type as key]: Dictionary[First Name as key]: BasicStats for that Individual that day
    '''
    
    DayWiseDistribution = getDayWiseDataDistribution(chatDataList)
    
    DayMemberBasicStats = {}
    
    for key in DayWiseDistribution.keys():
        DayMemberBasicStats[key] = getIndividualBasicStats(DayWiseDistribution[key])
        
    return DayMemberBasicStats

In [13]:
def extractDomainName(line):
    '''This function extracts and returns domain name of url 
    from the given sentence passed as parameter
    If no url is present, it returns None'''
    
    link = re.search("https://(.+?)/", line)  #if url is present, extracting till first '/' after https
    
    if link is None:
        return None
    
    link = link.group(1)
    
    linkWords = link.split('.')
    
    if len(linkWords) > 2:                     #first word mostly like is www
        return linkWords[1]
    else:
        return linkWords[0]                    #first word itself is the domain name

In [14]:
def getAllLinksStat(filename):
    '''This function takes filename as input parameter
    and returns a map with domain name as key & count as its value'''
    
    allChatDataSimplified = getSimplifiedChatData(filename)   #Getting simplified data
    allLinksStat = {}                                          #variable to store all links count
    
    for eachChatData in allChatDataSimplified:
        message = eachChatData[3]  
        
        domainName = extractDomainName(message)
        
        if domainName is not None:
            if domainName in allLinksStat:
                allLinksStat[domainName] = allLinksStat[domainName] + 1
            else:
                allLinksStat[domainName] = 1
    
    return allLinksStat

In [39]:
def getDayWiseStatPlots(chatDataList, plots = True):
    '''
    Input: List output from getSimplifiedChatData function, plots (True for plot outputs, false for only Data)
    Output: -tuple of following lists (Dates, nMsgEachDay, nCharactersEachDay, nWordsEachDay, nMediaEachDay,
                                      CharPerMsgEachDay, LenLongestMsgForDay, WordsPerMsgForDay)
            -Saves the plots of all elements in list (except Dates) vs Dates plots in directory:
            <current location>/Plots/DayWisePlots/                                
    '''
    
    DayWiseStats = getDayWiseBasicStats(chatDataList)
    
    nMsgEachDay = []
    nCharactersEachDay = []
    nWordsEachDay = []
    nMediaEachDay = []
    CharPerMsgEachDay = []
    LenLongestMsgForDay = []
    WordsPerMsgForDay = []
    Dates = list(DayWiseStats.keys())
        
    for key in DayWiseStats.keys():
        (nMsg, _, nCharacters, nWords, nMedia, _, AvgCharPerMsg, _, LenLongestMsg, AvgWordsPerMsg, _, _) = DayWiseStats[key]
        nMsgEachDay.append(nMsg)
        nCharactersEachDay.append(nCharacters)
        nWordsEachDay.append(nWords)
        nMediaEachDay.append(nMedia)
        CharPerMsgEachDay.append(AvgCharPerMsg)
        LenLongestMsgForDay.append(LenLongestMsg)
        WordsPerMsgForDay.append(AvgWordsPerMsg)
        
        
    
    # Plots
    if plots == True:

        if not os.path.exists('Plots/IndividualPlots'):
            os.makedirs('Plots/DayWisePlots')

        fig_size = 10

        plt.figure(figsize = (fig_size, fig_size))
        plt.plot(Dates, nMsgEachDay)
        plt.xlabel('Dates')
        plt.ylabel('No. of Message Each Day')
        plt.savefig('Plots/DayWisePlots/nMsgVsDay.jpg')

        plt.figure(figsize = (fig_size, fig_size))
        plt.plot(Dates, nCharactersEachDay)
        plt.xlabel('Dates')
        plt.ylabel('No. of Characters Each Day')
        plt.savefig('Plots/DayWisePlots/nCharactersvsDay.jpg')

        plt.figure(figsize = (fig_size, fig_size))
        plt.plot(Dates, nWordsEachDay)
        plt.xlabel('Dates')
        plt.ylabel('No. of Words Each Day')
        plt.savefig('Plots/DayWisePlots/nWordsVsDay.jpg')

        plt.figure(figsize = (fig_size, fig_size))
        plt.plot(Dates, nMediaEachDay)
        plt.xlabel('Dates')
        plt.ylabel('No. of Media Each Day')
        plt.savefig('Plots/DayWisePlots/nMediaVsDay.jpg')

        plt.figure(figsize = (fig_size, fig_size))
        plt.plot(Dates, CharPerMsgEachDay)
        plt.xlabel('Dates')
        plt.ylabel('No. of Characters per Message Each Day')
        plt.savefig('Plots/DayWisePlots/nCharPerMsgVsDay')

        plt.figure(figsize = (fig_size, fig_size))
        plt.plot(Dates, LenLongestMsgForDay)
        plt.xlabel('Dates')
        plt.ylabel('Length of longest Msg for that Day')
        plt.savefig('Plots/DayWisePlots/lenLongestMsgVsDay.jpg')

        plt.figure(figsize = (fig_size, fig_size))
        plt.plot(Dates, WordsPerMsgForDay)
        plt.xlabel('Dates')
        plt.ylabel('No. of Words per Message for that Day')
        plt.savefig('Plots/DayWisePlots/WordsPerMsgVsDay.jpg')

        
    return (Dates, nMsgEachDay, nCharactersEachDay, nWordsEachDay, nMediaEachDay, CharPerMsgEachDay,
            LenLongestMsgForDay, WordsPerMsgForDay)

In [35]:
def getIndividualStatsRadarPlot(chatDataList, plots = True):
    '''
    Input: List output from getSimplifiedChatData function, plots (True for plot outputs, false for only Data)
    Output: -tuple of list of categories for radar plot, dictionary with individual name as key
            and data corresponding to categories for that individual, max values among all participants
            for element in categories
            -Saves the Radar plot of elements in category list in directory: 
            <current location>/Plots/IndividualPlots/
    Caveats: -categories list is defined manually and few lines (marked) have to be changed if categories list is changed
    '''
    IndividualStats = getIndividualBasicStats(chatDataList)
    
    categories = ["nMsg", "nWords", "nMedia", "AvgMsgPerDay", "LenLongestMsg", "AvgWordsPerMsg"]
    categories = [*categories, categories[0]] #To avoid overlap between first and last category in Radar plot
    PlotData = {}
    
    
    # For Normalizing the Data
    # Needs to be changed if 'categories' list is changed
    
    nMsgMax = 0
    nWordsMax = 0
    nMediaMax = 0
    AvgMsgPerDayMax = 0
    LenLongestMsgMax = 0
    AvgWordsPerMsgMax = 0
    
    for key in IndividualStats.keys(): # Needs to be changed if 'categories' list is changed
        (nMsg, _, _, nWords, nMedia, AvgMsgPerDay, _,
         _, LenLongestMsg, AvgWordsPerMsg, _, _) = IndividualStats[key] 
        
        nMsgMax = max(nMsgMax, nMsg)
        nWordsMax = max(nWordsMax, nWords)
        nMediaMax = max(nMediaMax, nMedia)
        AvgMsgPerDayMax = max(AvgMsgPerDayMax, AvgMsgPerDay)
        LenLongestMsgMax = max(LenLongestMsgMax, LenLongestMsg)
        AvgWordsPerMsgMax = max(AvgWordsPerMsgMax, AvgWordsPerMsg)
    
    # Needs to be changed if 'categories' list is changed
    max_values = [nMsgMax, nWordsMax, nMediaMax, AvgMsgPerDayMax, LenLongestMsgMax, AvgWordsPerMsgMax]
    max_values = [*max_values, max_values[0]]
    
    
    for key in IndividualStats.keys(): # Needs to be changed if 'categories' list is changed
        (nMsg, _, _, nWords, nMedia, AvgMsgPerDay, _,
         _, LenLongestMsg, AvgWordsPerMsg, _, _) = IndividualStats[key]
        
        PlotData[key] = [nMsg/nMsgMax, nWords/nWordsMax, nMedia/nMediaMax, AvgMsgPerDay/AvgMsgPerDayMax,
                         LenLongestMsg/LenLongestMsgMax, AvgWordsPerMsg/AvgWordsPerMsgMax]
        PlotData[key] = [*PlotData[key], PlotData[key][0]] #To avoid overlap between first and last category in Radar plot
        
        
    
    # Plots
    if plots == True:

        if not os.path.exists('Plots/IndividualPlots'):
            os.makedirs('Plots/IndividualPlots')

        label_loc = np.linspace(start=0, stop=2 * np.pi, num=len(categories))
        fig_size = 12
        plt.figure(figsize = (fig_size, fig_size))
        plt.subplot(polar=True)

        for key in PlotData.keys():
            plt.plot(label_loc, PlotData[key], label=key)

        plt.title('Normalized Individual Stats', size=20)
        lines, labels = plt.thetagrids(np.degrees(label_loc), labels=categories)
        plt.legend()
        plt.savefig('Plots/IndividualPlots/RadarPlot.jpg')

    
    
    return (categories, PlotData, max_values)
    
    
    
    
    

In [17]:
Data = getSimplifiedChatData(fileName)

In [36]:
RadarPlotData = getIndividualStatsRadarPlot(Data, False)

In [82]:
len(RadarPlotData[1]['Apoorv'])

7

In [83]:
len(RadarPlotData[0])

7

True

In [41]:
b = getDayWiseStatPlots(Data, False)

In [ ]:
a = getIndividualStatsRadarPlot(Data)